In [2]:
#import der benötigten Python Libraries 
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import time
import datetime

import re
import emoji

import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 500)

#um die häufigsten Wörter im Wahlkampf anzeigen lassen zu können > wird zur tokenisierung etc benötigt
import nltk 
from nltk.corpus import stopwords
from nltk import FreqDist

import spacy

In [3]:
#DataFrames der CDU
df_cdu_raw = pd.read_excel("../data/Parteien/@CDU_user_tweets_bearbeitet.xlsx")
df_cdu_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
# Konverteiren des DataSets + umbenennen der Spalte
tweets_df_clean = df_cdu_raw[['Name', 'original_tweets']]


In [168]:
import string 

def clean_text(text):
#konvertiert text zu lowercase  
 text = text.lower()
#löscht text in eckigen Klammern
 text = re.sub(r'\[.*?\]', '', text)
#löscht Punktuation + Links + @Zeichen 
 text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
#löscht Wörter die Nummern enthalten
 text = re.sub(r'\w*\d\w*', '', text)
#löscht Anführungsstriche
 text = re.sub(r'„', '', text)
 text = re.sub(r'“', '', text)
 return text

def give_emoji_free_text(text): #Emojis werden aus den tweets entfernt 
    return emoji.get_emoji_regexp().sub(r'',text)

tweets_df_clean['emoji_free_tweets'] = tweets_df_clean['original_tweets'].apply(give_emoji_free_text)

#Neue Spalte für Text ohne Emojis
tweets_df_clean['clean_tweets'] = tweets_df_clean['emoji_free_tweets'].apply(clean_text)                               
                               
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets
0,CDU Deutschlands,".@ArminLaschet in der #BerlinerRunde: „Die Zeit die vor uns liegt, braucht als erstes wirtschaftliche Stärke &amp; Stabilität, marktwirtschaftliche Lösungen und einen großen Schub in der Verwaltung des Landes, der Digitalisierung sowie im Klimaschutz!“",".@ArminLaschet in der #BerlinerRunde: „Die Zeit die vor uns liegt, braucht als erstes wirtschaftliche Stärke &amp; Stabilität, marktwirtschaftliche Lösungen und einen großen Schub in der Verwaltung des Landes, der Digitalisierung sowie im Klimaschutz!“",arminlaschet in der berlinerrunde die zeit die vor uns liegt braucht als erstes wirtschaftliche stärke amp stabilität marktwirtschaftliche lösungen und einen großen schub in der verwaltung des landes der digitalisierung sowie im klimaschutz
1,CDU Deutschlands,Jetzt live: Statement von @ArminLaschet am Wahlabend der #btw21.\n https://t.co/BjbG8lb8aG,Jetzt live: Statement von @ArminLaschet am Wahlabend der #btw21.\n https://t.co/BjbG8lb8aG,jetzt live statement von arminlaschet am wahlabend der \n
2,CDU Deutschlands,"Wir danken allen Wählerinnen und Wählern, die uns heute ihr Vertrauen geschenkt haben.","Wir danken allen Wählerinnen und Wählern, die uns heute ihr Vertrauen geschenkt haben.",wir danken allen wählerinnen und wählern die uns heute ihr vertrauen geschenkt haben
3,CDU Deutschlands,"An alle Wahlkämpferinnen und Wahlkämpfer, die bei Regen, Sonne oder Sturm, bei Tag und Nacht, von Tür zu Tür, auf den Marktplätzen oder in den sozialen Medien für eine starke CDU und @ArminLaschet als Bundeskanzler gekämpft haben.","An alle Wahlkämpferinnen und Wahlkämpfer, die bei Regen, Sonne oder Sturm, bei Tag und Nacht, von Tür zu Tür, auf den Marktplätzen oder in den sozialen Medien für eine starke CDU und @ArminLaschet als Bundeskanzler gekämpft haben.",an alle wahlkämpferinnen und wahlkämpfer die bei regen sonne oder sturm bei tag und nacht von tür zu tür auf den marktplätzen oder in den sozialen medien für eine starke cdu und arminlaschet als bundeskanzler gekämpft haben
4,CDU Deutschlands,"Es ist ein Kopf-an-Kopf-Rennen und wird ein langer Wahlabend. Zum jetzigen Zeitpunkt kann man noch nicht sagen, wer am Ende die Nase vorn haben wird.\nJetzt möchten wir erst einmal Danke sagen: https://t.co/G87SxsFhQd","Es ist ein Kopf-an-Kopf-Rennen und wird ein langer Wahlabend. Zum jetzigen Zeitpunkt kann man noch nicht sagen, wer am Ende die Nase vorn haben wird.\nJetzt möchten wir erst einmal Danke sagen: https://t.co/G87SxsFhQd",es ist ein kopfankopfrennen und wird ein langer wahlabend zum jetzigen zeitpunkt kann man noch nicht sagen wer am ende die nase vorn haben wird\njetzt möchten wir erst einmal danke sagen
...,...,...,...,...
588,CDU Deutschlands,"Die Vernetzung der Gamesbranche mit anderen Wirtschaft- und Industriezweigen, dafür steht das heute von @ArminLaschet eröffnete German Center of Games Competence.","Die Vernetzung der Gamesbranche mit anderen Wirtschaft- und Industriezweigen, dafür steht das heute von @ArminLaschet eröffnete German Center of Games Competence.",die vernetzung der gamesbranche mit anderen wirtschaft und industriezweigen dafür steht das heute von arminlaschet eröffnete german center of games competence
589,CDU Deutschlands,#servicetweet - um 10:15 eröffnet @ArminLaschet den Fusion Campus! https://t.co/SCiTtE0OR7,#servicetweet - um 10:15 eröffnet @ArminLaschet den Fusion Campus! https://t.co/SCiTtE0OR7,servicetweet um eröffnet arminlaschet den fusion campus
590,CDU Deutschlands,"„Außenpolitik besteht darin, mit Staaten, die völlig anders denken als wir, Gemeinsamkeiten zu definieren und zu verändern.“ @ArminLaschet bei der #MSCdebate https://t.co/7J3hUnZNMs","„Außenpolitik besteht darin, mit Staaten, die völlig anders denken als wir, Gemeinsamkeiten zu definieren und zu verändern.“ @ArminLaschet bei der #MSCdebate https://t.co/7J3hUnZNMs",außenpolitik besteht darin mit staaten die völlig anders denken als wir gemeinsamkeiten zu definieren u

In [169]:
data = list(tweets_df_clean.loc[:, "clean_tweets"].values) ## data ist ein doc

In [170]:
# Die Wörter werden lemmatisiert -> ihr Ursprungsform wird betracht (ungebeugt)
# durch den Parameter "allowed_postags" filtern wir den Text und lassen nur Substatntive und Adjektive ausgeben
def lemmatization(texts, allowed_postags=["NOUN","ADJ"]):
    nlp = spacy.load("de_core_news_lg", disable=["parser", "ner"]) 
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:120])

berlinerrunde zeit erst wirtschaftlich stärken stabilität marktwirtschaftliche lösungen groß schub verwaltung landes dig


In [171]:
tweets_df_clean['clean_tweets_noun'] = lemmatized_texts

In [172]:
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets,clean_tweets_noun
0,CDU Deutschlands,".@ArminLaschet in der #BerlinerRunde: „Die Zeit die vor uns liegt, braucht als erstes wirtschaftliche Stärke &amp; Stabilität, marktwirtschaftliche Lösungen und einen großen Schub in der Verwaltung des Landes, der Digitalisierung sowie im Klimaschutz!“",".@ArminLaschet in der #BerlinerRunde: „Die Zeit die vor uns liegt, braucht als erstes wirtschaftliche Stärke &amp; Stabilität, marktwirtschaftliche Lösungen und einen großen Schub in der Verwaltung des Landes, der Digitalisierung sowie im Klimaschutz!“",arminlaschet in der berlinerrunde die zeit die vor uns liegt braucht als erstes wirtschaftliche stärke amp stabilität marktwirtschaftliche lösungen und einen großen schub in der verwaltung des landes der digitalisierung sowie im klimaschutz,berlinerrunde zeit erst wirtschaftlich stärken stabilität marktwirtschaftliche lösungen groß schub verwaltung landes digitalisierung klimaschutz
1,CDU Deutschlands,Jetzt live: Statement von @ArminLaschet am Wahlabend der #btw21.\n https://t.co/BjbG8lb8aG,Jetzt live: Statement von @ArminLaschet am Wahlabend der #btw21.\n https://t.co/BjbG8lb8aG,jetzt live statement von arminlaschet am wahlabend der \n,statement
2,CDU Deutschlands,"Wir danken allen Wählerinnen und Wählern, die uns heute ihr Vertrauen geschenkt haben.","Wir danken allen Wählerinnen und Wählern, die uns heute ihr Vertrauen geschenkt haben.",wir danken allen wählerinnen und wählern die uns heute ihr vertrauen geschenkt haben,wählerinnen wählern vertrauen
3,CDU Deutschlands,"An alle Wahlkämpferinnen und Wahlkämpfer, die bei Regen, Sonne oder Sturm, bei Tag und Nacht, von Tür zu Tür, auf den Marktplätzen oder in den sozialen Medien für eine starke CDU und @ArminLaschet als Bundeskanzler gekämpft haben.","An alle Wahlkämpferinnen und Wahlkämpfer, die bei Regen, Sonne oder Sturm, bei Tag und Nacht, von Tür zu Tür, auf den Marktplätzen oder in den sozialen Medien für eine starke CDU und @ArminLaschet als Bundeskanzler gekämpft haben.",an alle wahlkämpferinnen und wahlkämpfer die bei regen sonne oder sturm bei tag und nacht von tür zu tür auf den marktplätzen oder in den sozialen medien für eine starke cdu und arminlaschet als bundeskanzler gekämpft haben,wahlkämpferinnen wahlkämpfer regen sonne sturm tag nacht tür tür marktplätzen sozial medien starke bundeskanzler
4,CDU Deutschlands,"Es ist ein Kopf-an-Kopf-Rennen und wird ein langer Wahlabend. Zum jetzigen Zeitpunkt kann man noch nicht sagen, wer am Ende die Nase vorn haben wird.\nJetzt möchten wir erst einmal Danke sagen: https://t.co/G87SxsFhQd","Es ist ein Kopf-an-Kopf-Rennen und wird ein langer Wahlabend. Zum jetzigen Zeitpunkt kann man noch nicht sagen, wer am Ende die Nase vorn haben wird.\nJetzt möchten wir erst einmal Danke sagen: https://t.co/G87SxsFhQd",es ist ein kopfankopfrennen und wird ein langer wahlabend zum jetzigen zeitpunkt kann man noch nicht sagen wer am ende die nase vorn haben wird\njetzt möchten wir erst einmal danke sagen,kopfankopfrennen lang jetzig zeitpunkt ende nase danke
...,...,...,...,...,...
588,CDU Deutschlands,"Die Vernetzung der Gamesbranche mit anderen Wirtschaft- und Industriezweigen, dafür steht das heute von @ArminLaschet eröffnete German Center of Games Competence.","Die Vernetzung der Gamesbranche mit anderen Wirtschaft- und Industriezweigen, dafür steht das heute von @ArminLaschet eröffnete German Center of Games Competence.",die vernetzung der gamesbranche mit anderen wirtschaft und industriezweigen dafür steht das heute von arminlaschet eröffnete german center of games competence,vernetzung gamesbranche ander wirtschaft industriezweigen
589,CDU Deutschlands,#servicetweet - um 10:15 eröffnet @ArminLaschet den Fusion Campus! https://t.co/SCiTtE0OR7,#servicetweet - um 10:15 eröffnet @ArminLaschet den Fusion Campus! https://t.co/SCiTtE0OR7,servicetweet um eröffnet arminlaschet den fusion campus,servicetweet fusion
590,CDU Deutschlands,"„Außenpolitik bes

In [173]:
#stopwords werden entfernt 
#sätze werden in einzelne Wörter heruntergebochen 

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('german')
stop_words.extend(['amp','mehr erfahren','erfahren','arminlaschet', 'wegenmorgen'])
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = tweets_df_clean.clean_tweets_noun.values.tolist()
data_words = list(sent_to_words(data))
# Stopwords entfernen
data_words = remove_stopwords(data_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philipp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [174]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(tweets_df_clean.clean_tweets_noun, 10)
unigram = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

In [175]:
unigram

,unigram,count
0,gut,48
1,land,47
2,menschen,43
3,wachstum,37
4,krise,36
5,sicherheit,33
6,jobs,30
7,eineunion,29
8,wirtschaft,29
9,sicher,29


In [176]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2,2),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(tweets_df_clean.clean_tweets_noun, 10)
bigram = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

In [177]:
bigram

,bigram,count
0,klimaneutralen industrieland,11
1,gut sicher,11
2,sicher jobs,11
3,soldatinnen soldaten,10
4,innovationen technologien,8
5,wachstum gut,8
6,agenda sicher,8
7,kampf klimawandel,7
8,nachhaltig wachstum,7
9,klimaneutrales industrieland,7


In [178]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3,3),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(tweets_df_clean.clean_tweets_noun, 10)
trigram = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

In [179]:
trigram

,trigram,count
0,gut sicher jobs,10
1,wachstum gut sicher,7
2,klimaschutz wirtschaftswachstum ziel,5
3,wirtschaftswachstum ziel innovationen,5
4,ziel innovationen technologien,5
5,innovationen technologien wachstum,5
6,technologien wachstum gut,5
7,klimaneutralen industrieland klimaschutz,4
8,industrieland klimaschutz wirtschaftswachstum,4
9,wachstum wettbewerbsfähigkeit planungsbeschleunigungspaket,4


In [180]:
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(
                        analyzer='word', 
                        ngram_range=(1,2),#bigramme werden berücksichtigt 
                        max_df=0.8,
                        min_df=2,# minimales Vorkommen eines Wortes
                        stop_words= stop_words,#stopwords werden entfernt 
                        lowercase=True #lowercase 
                        )
data_matrix = vectorizer.fit_transform(tweets_df_clean.clean_tweets_noun)
data_matrix

<593x900 sparse matrix of type '<class 'numpy.int64'>'
	with 3826 stored elements in Compressed Sparse Row format>

In [189]:
lda_model = LatentDirichletAllocation(
n_components=5, # Nummer von Topics
learning_method='batch',
random_state=25,       
n_jobs = -1  # Alle verfügbaren CPUs werden verwendet 
                                     )
lda_output = lda_model.fit_transform(data_matrix)

In [190]:
for i,topic in enumerate(lda_model.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')


Top 10 words for topic #0:
['arbeitsplätze', 'euro', 'bund', 'union', 'steuern', 'steuererhöhungen', 'neue', 'wirtschaft', 'pandemie', 'krise']


Top 10 words for topic #1:
['neu', 'stark', 'familien', 'uhr', 'triell', 'sofortprogramm', 'ideen', 'bundestagswahl', 'industrieland', 'land']


Top 10 words for topic #2:
['stimmen', 'gut', 'sicher', 'klartext', 'zukunftsteam', 'land', 'zukunft', 'sicherheit', 'eineunion', 'menschen']


Top 10 words for topic #3:
['welt', 'grün', 'veto', 'blick', 'kampf', 'klimawandel', 'klartext', 'schlussrunde', 'turbo', 'olafscholz']


Top 10 words for topic #4:
['technologien', 'innovationen', 'nachhaltig', 'gut sicher', 'sicher jobs', 'sicher', 'klimaschutz', 'wachstum', 'gut', 'jobs']




In [99]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      150.222153  -72.269554       1        1  27.751365
1       51.163364 -159.968018       2        1  21.514932
2      -73.144554   21.523815       3        1  20.343841
4      -66.410133 -110.606438       4        1  15.649390
0       54.417324   12.443444       5        1  14.740472, topic_info=              Term       Freq      Total Category  logprob  loglift
407       wachstum  33.000000  33.000000  Default  30.0000  30.0000
165           jobs  27.000000  27.000000  Default  29.0000  29.0000
241            neu  17.000000  17.000000  Default  28.0000  28.0000
201          krise  32.000000  32.000000  Default  27.0000  27.0000
207           land  41.000000  41.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
229         modern   4.499961  12.926405   Topic5  -4.5944   0.8594
243           neue   4.685925  24.628695   Topic5  -4.5539   0.2552
181       klartext   3.820140  25.090451   Topic5  -4.7582   0.0324
42   bundeskanzler   3.636752   7.871603   Topic5  -4.8073   1.1424
302   schlussrunde   3.632469  14.780814   Topic5  -4.8085   0.5112

[282 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
2         5  0.924398      abschwung
3         1  0.209578         agenda
3         2  0.733524         agenda
4         2  0.904720  agenda sicher
9         3  0.720491  arbeitsplätze
...     ...       ...            ...
448       5  0.139644        zukunft
449       1  0.250602   zukunftsteam
449       2  0.100241   zukunftsteam
449       3  0.651564   zukunftsteam
451       3  0.930374   zusammenhalt

[370 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 5, 1])